In [211]:
import requests
import sqlite3
import bs4
import pandas as pd

def get_index_table():
    url = 'https://www.hl7.org/fhir/resourcelist.html'
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, 'html.parser')
    table = bs4.BeautifulSoup(str(soup.find_all('table')))
    return table

def resource_categories_dict(table):
    column_names = table.find_all(class_ = 'frm-group')
    count = 0
    grouping_category = {}
    for column in column_names:
        column_values = column.find_all(class_ = "frm-category")
        column_group = column.find(class_ = "frm-group rotate")
        if column_values != []:
            count += 1
            element_count = 0
            grouping_category.update({"group-"+str(count): column_group.text})
            for temp_value in column_values:
                grouping_category.update({"category-"+str(element_count)+"-"+str(count): temp_value.text})
                element_count += 1    
    return grouping_category

def resources_list(grouping_category, 
                   table, 
                   connection):
    column_content = table.find_all(class_ = 'frm-contents')
    create_resource_table(c)
    resources_csv = []
    for group_count in range(0,5):
        category_count = 0
        for resource_tile in column_content[group_count]:
            if isinstance(resource_tile, bs4.element.Tag):
                list_resource = resource_tile.find_all('li')
                for resource in list_resource:
                    resource = resource.find_all('a')
                    for temp_resource in resource:
                        if temp_resource['title'] != "Maturity Level" and temp_resource.text != "N":
                            temp_group_count = grouping_category["group-"+str(group_count+1)]
                            temp_category_count = grouping_category["category-"+str(category_count)+"-"+str(group_count+1)]
                            temp_resource_name = temp_resource.text
                            temp_resource_description = temp_resource['title']
                            temp_resource_description = temp_resource_description.replace("'", "")
                            temp_resource_url = "https://www.hl7.org/fhir/" + temp_resource['href']
                            resources_csv.append([temp_group_count, 
                                            temp_category_count, 
                                            temp_resource_name,
                                            temp_resource_description,
                                            temp_resource_url])
                            create_data_structure_row(temp_group_count, 
                                            temp_category_count, 
                                            temp_resource_name,
                                            temp_resource_description,
                                            temp_resource_url,
                                            connection)
                category_count += 1
    resources_csv = pd.DataFrame(resources_csv)
    resources_csv.to_csv("Resources.csv")
        

def create_data_structure_row(temp_group_count, 
                              temp_category_count, 
                              temp_resource_name,
                              temp_resource_description,
                              temp_resource_url,
                              c):
    insert_string = "INSERT INTO 'FHIR-Resources-List' VALUES ('"
    insert_string += str(temp_group_count) + "','" + str(temp_category_count) + "','" + str(temp_resource_name) + "','"   
    insert_string += str(temp_resource_description)   + "','" +  str(temp_resource_url) +"')"
    c.execute(insert_string)
                  
def create_resource_table(c):
    c.execute("DROP TABLE IF EXISTS 'FHIR-Resources-List'")
    create_string = "CREATE TABLE 'FHIR-Resources-List' "  
    create_string += """(Grouping TEXT,
                          Category TEXT,
                          Name TEXT,
                          Description TEXT,
                          URL TEXT
                        )"""
    c.execute(create_string)

database_directory = './FHIR-Resources.db'
conn = sqlite3.connect(database_directory)
c = conn.cursor()

resource_table = get_index_table()
grouping_category = resource_categories_dict(resource_table)         
resources_list(grouping_category, resource_table, c)

conn.commit()
conn.close()
        

In [231]:
import requests
import bs4
def get_resource_table(resource_url):
    r = requests.get(resource_url)
    soup = bs4.BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all("table")
    for element in table:
        print(type(element))
        #print(element)
        resources = element.find_all("a")
        for resource in resources: 
            try:
                print(resource['href'])
                print("-----------------------------")
            except:
                continue
                
        print("-----------------------------------------------------------------------------")
    return table

resource_url = "https://www.hl7.org/fhir/allergyintolerance.html"
table = get_resource_table(resource_url)


<class 'bs4.element.Tag'>
http://www.hl7.org/Special/committees/patientcare/index.cfm
-----------------------------
versions.html#maturity
-----------------------------
versions.html#std-process
-----------------------------
security.html#SecPrivConsiderations
-----------------------------
compartmentdefinition.html
-----------------------------
compartmentdefinition-patient.html
-----------------------------
compartmentdefinition-practitioner.html
-----------------------------
compartmentdefinition-relatedperson.html
-----------------------------
-----------------------------------------------------------------------------
<class 'bs4.element.Tag'>
formats.html#table
-----------------------------
formats.html#table
-----------------------------
formats.html#table
-----------------------------
formats.html#table
-----------------------------
formats.html#table
-----------------------------
formats.html#table
-----------------------------
allergyintolerance-definitions.html#AllergyIntol